In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
enfermedades = pd.read_excel('archivos/CIE.xlsx')
new_df=  pd.read_csv ('archivos/NewDataAll.csv')

In [3]:
new_df['Caudef']
enfermedades2 = enfermedades.copy()
enfermedades2.loc[enfermedades2['CAUSA'].str.len() == 3, 'CAUSA'] = enfermedades2.loc[enfermedades2['CAUSA'].str.len() == 3, 'CAUSA'] + "X"
enfermedades2

new_df2 = pd.merge(new_df, enfermedades2, left_on='Caudef', right_on='CAUSA', how='left')

new_df2 = new_df2.dropna()
causas_infecciones = new_df2.loc[new_df2["DESCRIP"].str.contains("Infecciones|infecciones|infeccion|Infección|Infeccion|infección")]

causas_vih = new_df2.loc[new_df2["DESCRIP"].str.contains("VIH|vih|SIDA")]


In [4]:
new_df2['VIH'] = 0
indices_vih = new_df2[new_df2['DESCRIP'] .str.contains("VIH|vih|SIDA")].index
new_df2.loc[indices_vih, 'VIH'] = 1


In [5]:
new_df2.loc[new_df2["DESCRIP"].str.contains("VIH|vih|SIDA")]


,Unnamed: 0,Depreg,Mupreg,Mesreg,Añoreg,Depocu,Mupocu,Areag,Sexo,Diaocu,...,Nacdif,Dredif,Mredif,Caudef,Asist,Ocur,Cerdef,CAUSA,DESCRIP,VIH
448,448,1,101,11,9,1,101,1,1,13,...,0,1,101,B24X,1,1,1,B24X,Enfermedad por virus de la inmunodeficiencia h...,1
679,679,1,101,4,9,1,101,1,2,29,...,0,1,101,B207,1,1,1,B207,"Enfermedad por VIH, resultante en infecciones ...",1
705,705,1,101,3,9,1,101,1,1,28,...,0,1,101,B24X,1,1,1,B24X,Enfermedad por virus de la inmunodeficiencia h...,1
735,735,1,101,5,9,1,101,1,2,5,...,0,1,101,B200,1,1,1,B200,"Enfermedad por VIH, resultante en infección po...",1
845,845,1,101,6,9,1,101,1,1,31,...,0,1,101,B24X,1,1,1,B24X,Enfermedad por virus de la inmunodeficiencia h...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70970,70970,4,401,10,9,4,401,1,2,26,...,0,99,9999,B24X,1,1,2,B24X,Enfermedad por virus de la inmunodeficiencia h...,1
71597,71597,1,101,8,9,1,101,1,1,17,...,99,99,9999,B208,1,1,1,B208,"Enfermedad por VIH, resultante en otras enferm...",1
71625,71625,5,501,10,9,5,501,1,1,13,...,99,99,9999,B24X,1,1,1,B24X,Enfermedad por virus de la inmunodeficiencia h...,1
71628,71628,18,1801,5,9,18,1801,1,2,30,...,0,99,9999,B206,1,1,1,B206,"Enfermedad por VIH, resultante en neumonía por...",1


In [15]:
# Filtrar las características y la variable objetivo
#X = new_df2[['Depreg', 'Mupreg', 'Mesreg', 'Añoreg', 'Depocu', 'Mupocu', 'Sexo', 'Diaocu', 'Mesocu', 'Añoocu', 'Edadif', 'Perdif', 'Ecidif', 'Dnadif', 'Nacdif', 'Dredif', 'Mredif', 'Asist', 'Ocur', 'Cerdef']]
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE

X = new_df2[['Edadif']]
y = new_df2['VIH']
# Balancear las clases utilizando SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [ ]:
# Crear una instancia del modelo SVM y entrenarlo con diferentes parámetros
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test)

# Calcular la precisión (accuracy), sensibilidad (sensitivity) y especificidad (specificity)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

true_negative = confusion[0, 0]
false_positive = confusion[0, 1]
false_negative = confusion[1, 0]
true_positive = confusion[1, 1]

sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

# Imprimir los resultados
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
